In [ ]:
import os
from youtube_dl import YoutubeDL
import speech_recognition as sr
import soundfile

In [ ]:
audio_downloader = YoutubeDL({'format':'bestaudio','outtmpl': 'voice.mp4'})

while True:
  try:
    print('Youtube Downloader'.center(40, '_'))
    URL = input('Enter youtube url :  ')
    audio_downloader.extract_info(URL)
  except Exception:
    print("Couldn\'t download the audio")
  break

In [ ]:
data, samplerate = soundfile.read('out.wav')
soundfile.write('new.wav', data, samplerate, subtype='PCM_16')


In [ ]:
r = sr.Recognizer() 
with sr.AudioFile("out.wav") as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    # speech recognition 
    result = r.recognize_google()
    # export the result 
    with open('my_result.txt',mode ='w') as file: 
        file.write("Recognized text:") 
        file.write("\n") 
        file.write(result) 
        print("Exporting process completed!")

In [ ]:

from vosk import Model, KaldiRecognizer
import json
import os

model = Model("eng")

# Large vocabulary free form recognition
rec = KaldiRecognizer(model, 16000)

# You can also specify the possible word list
#rec = KaldiRecognizer(model, 16000, "zero oh one two three four five six seven eight nine")

wf = open("voice.mp4", "rb")
wf.read(44) # skip header

while True:
    data = wf.read(4000)
    if len(data) == 0:
        break
    if rec.AcceptWaveform(data):
        res = json.loads(rec.Result())
        print (res['text'])

res = json.loads(rec.FinalResult())
print(res)